In [1]:
import pandas as pd
import numpy as np
import calendar
import datetime as dt
import scipy.stats as sps
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
from matplotlib import pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_30 = pd.read_csv('30_features.csv', index_col=0)
data_30

,fulladdress_count_30,address_count_14,fulladdress_pct_increase_0_to_30,fulladdress_count_0_by_30,address_count_3,fulladdress_count_0_by_7,fulladdress_pct_increase_0_to_7,address_count_1,fulladdress_count_1,fulladdress_homephone_count_30,...,ssn_name_dob_count_14,ssn_firstname_count_0_by_30,ssn_firstname_pct_increase_0_to_30,ssn_lastname_count_0_by_30,fulladdress_homephone_pct_increase_0_to_30,fulladdress_homephone_count_0_by_30,address_pct_increase_1_to_14,dob_count_wSame_homephone_7,name_count_wSame_homephone_7,fraud_label
0,1,1,0.0,30.0,1,7.0,0.0,1,1,1,...,1,30.0,0.0,30.0,0.0,30.0,0.0,2,2,0
1,1,1,0.0,30.0,1,7.0,0.0,1,1,1,...,1,30.0,0.0,30.0,0.0,30.0,0.0,2,2,1
2,1,1,0.0,30.0,1,7.0,0.0,1,1,1,...,1,30.0,0.0,30.0,0.0,30.0,0.0,1,1,0
3,1,1,0.0,30.0,1,7.0,0.0,1,1,1,...,1,30.0,0.0,30.0,0.0,30.0,0.0,1,1,0
4,1,1,0.0,30.0,1,7.0,0.0,1,1,1,...,1,30.0,0.0,30.0,0.0,30.0,0.0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,1,1,0.0,30.0,1,7.0,0.0,1,1,1,...,1,30.0,0.0,30.0,0.0,30.0,0.0,1,1,0
999996,1,1,0.0,30.0,1,7.0,0.0,1,1,1,...,1,30.0,0.0,30.0,0.0,30.0,0.0,2,2,0
999997,1,1,0.0,30.0,1,7.0,0.0,1,1,1,...,1,30.0,0.0,30.0,0.0,30.0,0.0,2,2,0
999998,1,1,0.0,30.0,1,7.0,0.0,1,1,1,...,1,30.0,0.0,30.0,0.0,30.0,0.0,4,4,0


## Split data to oot & Train_test

In [4]:
oot=data_30.iloc[833507:1000000,:]
X_oot=oot.drop('fraud_label',axis=1)
y_oot=oot.fraud_label

train_test=data_30.iloc[38511:833507,:]
X=train_test.drop('fraud_label',axis=1)
y=train_test.fraud_label

In [5]:
print('oot_X: ',X_oot.shape,
      '\noot_y: ',y_oot.shape,
      '\nX: ',X.shape,
      '\ny: ',y.shape)

oot_X:  (166493, 30) 
oot_y:  (166493,) 
X:  (794996, 30) 
y:  (794996,)


## Get array & scale

In [6]:
X_oot=X_oot.values
y_oot=y_oot.values
X=X.values
y=y.values

stdsc = StandardScaler()
X_std = stdsc.fit_transform(X)
X_oot_std = stdsc.fit_transform(X_oot)

## Apply Model: Logistic Regression

In [55]:
# if you don't want some intermediate printing,
# comment those out

def LG_train_test_oot_fdr(classifier, N=10, var_num=30):
    global X_std, y, X_oot, y_oot
    skf = StratifiedKFold(n_splits=N)

    FDR_train=np.zeros(N)
    FDR_test=np.zeros(N)
    FDR_oot=np.zeros(N)

    i = 0
    for train, test in skf.split(X_std, y):
        X_train, X_test, y_train, y_test = X_std[train][:,:var_num], X_std[test][:,:var_num], y[train], y[test]
        print(f'{i+1}\'th fold...')
        classifier.fit(X_train, y_train)
        print(f'\tFinal iteration: {classifier.n_iter_}')
        print(f'\tCoefficient: {classifier.coef_}')
        y_pred_tr = classifier.predict_proba(X_train)[:,1]
        y_pred_ts = classifier.predict_proba(X_test)[:,1]
        y_pred_oot = classifier.predict_proba(X_oot_std[:,:var_num])[:,1]

        train_temp=pd.DataFrame(columns=['y_train','y_pred_tr'])
        Rows=round(len(y_train)*0.03)
        train_temp['y_train']=y_train
        train_temp['y_pred_tr']=y_pred_tr
        train_temp.sort_values('y_pred_tr', ascending=False, inplace=True)
        tempR=train_temp.head(Rows)
        num=tempR.loc[:,'y_train']
        den=((y_train==1).sum())
        fdr_tr=sum(num)/den
        FDR_train[i]=fdr_tr

        test_temp=pd.DataFrame(columns=['y_test','y_pred_ts'])
        Rows=round(len(y_test)*0.03)
        test_temp['y_test']=y_test
        test_temp['y_pred_ts']=y_pred_ts
        test_temp.sort_values('y_pred_ts', ascending=False, inplace=True)
        tempR=test_temp.head(Rows)
        num=tempR.loc[:,'y_test']
        den=((y_test==1).sum())
        fdr_ts=sum(num)/den
        FDR_test[i]=fdr_ts

        oot_temp=pd.DataFrame(columns=['y_oot','y_pred_oot'])
        Rows=round(len(y_oot)*0.03)
        oot_temp['y_oot']=y_oot
        oot_temp['y_pred_oot']=y_pred_oot
        oot_temp.sort_values('y_pred_oot', ascending=False, inplace=True)
        tempR=oot_temp.head(Rows)
        num=tempR.loc[:,'y_oot']
        den=((y_oot==1).sum())
        fdr_oot=sum(num)/den
        FDR_oot[i]=fdr_oot

        print(f'\tFDR_train: {FDR_train[i]}')
        print(f'\tFDR_test: {FDR_test[i]}')
        print(f'\tFDR_oot: {FDR_oot[i]}\n')
        i += 1

    print(FDR_train.mean())
    print(FDR_test.mean())
    print(FDR_oot.mean())

In [57]:
# sample input and output
# feel free to experiement
classifier = LogisticRegression(penalty='l1', solver='saga', class_weight='balanced', max_iter=5000)
LG_train_test_oot_fdr(classifier, N=10, var_num=25)

1'th fold...
	Final iteration: [398]
	Coefficient: [[ 1.15884043 -0.0390071   0.33629633  0.35829205  0.22778502 -0.31417838
  -0.52458828 -0.04415598  0.53155525  0.05968209  0.33914157 -1.44260274
   0.27073587  0.96000333  0.56051545  0.27540753 -0.1858      0.07044643
   0.22292628  0.60359236 -0.59069842  0.23567213  0.67956232  0.81549492
   0.28977078]]
	FDR_train: 0.5390345361323401
	FDR_test: 0.4691035683202785
	FDR_oot: 0.5125733445096395

2'th fold...
	Final iteration: [205]
	Coefficient: [[ 1.16086182 -0.03196982  0.34950503  0.37294724  0.28648943 -0.31299432
  -0.52638999 -0.11993637  0.54565253  0.17231995  0.34575149 -1.45392542
   0.28125859  0.95407949  0.40210814  0.29248341 -0.17203488 -0.04405704
   0.21721841  0.57645534 -0.57603856  0.36829678  0.66889894  0.82853401
   0.30122317]]
	FDR_train: 0.5389377962658412
	FDR_test: 0.46736292428198434
	FDR_oot: 0.5138306789606035

3'th fold...
	Final iteration: [307]
	Coefficient: [[ 1.00043533  0.00257846  0.38425409  0

## Apply Model: XGBoost

In [61]:
# if you don't want some intermediate printing,
# comment those out

def BT_train_test_oot_fdr(classifier, N=10, var_num=30):
    global X_std, y, X_oot, y_oot
    skf = StratifiedKFold(n_splits=N)

    FDR_train=np.zeros(N)
    FDR_test=np.zeros(N)
    FDR_oot=np.zeros(N)

    i = 0
    for train, test in skf.split(X_std, y):
        X_train, X_test, y_train, y_test = X_std[train][:,:var_num], X_std[test][:,:var_num], y[train], y[test]
        print(f'{i+1}th fold')
        classifier.fit(X_train, y_train)
        print(f'\tFeature Importance:\t\n{classifier.feature_importances_}')
        y_pred_tr = classifier.predict_proba(X_train)[:,1]
        y_pred_ts = classifier.predict_proba(X_test)[:,1]
        y_pred_oot = classifier.predict_proba(X_oot_std[:,:var_num])[:,1]

        train_temp=pd.DataFrame(columns=['y_train','y_pred_tr'])
        Rows=round(len(y_train)*0.03)
        train_temp['y_train']=y_train
        train_temp['y_pred_tr']=y_pred_tr
        train_temp.sort_values('y_pred_tr', ascending=False, inplace=True)
        tempR=train_temp.head(Rows)
        num=tempR.loc[:,'y_train']
        den=((y_train==1).sum())
        fdr_tr=sum(num)/den
        FDR_train[i]=fdr_tr

        test_temp=pd.DataFrame(columns=['y_test','y_pred_ts'])
        Rows=round(len(y_test)*0.03)
        test_temp['y_test']=y_test
        test_temp['y_pred_ts']=y_pred_ts
        test_temp.sort_values('y_pred_ts', ascending=False, inplace=True)
        tempR=test_temp.head(Rows)
        num=tempR.loc[:,'y_test']
        den=((y_test==1).sum())
        fdr_ts=sum(num)/den
        FDR_test[i]=fdr_ts

        oot_temp=pd.DataFrame(columns=['y_oot','y_pred_oot'])
        Rows=round(len(y_oot)*0.03)
        oot_temp['y_oot']=y_oot
        oot_temp['y_pred_oot']=y_pred_oot
        oot_temp.sort_values('y_pred_oot', ascending=False, inplace=True)
        tempR=oot_temp.head(Rows)
        num=tempR.loc[:,'y_oot']
        den=((y_oot==1).sum())
        fdr_oot=sum(num)/den
        FDR_oot[i]=fdr_oot
        print(f'\tFDR_train: {FDR_train[i]}')
        print(f'\tFDR_test: {FDR_test[i]}')
        print(f'\tFDR_oot: {FDR_oot[i]}\n')
        i += 1

    print(FDR_train.mean())
    print(FDR_test.mean())
    print(FDR_oot.mean())

In [123]:
# sample input and output
# feel free to experiement
classifier = XGBClassifier(subsample=0.8, max_depth=4, n_estimators=500, learning_rate=0.2, objective='binary:logistic', eval_metric='logloss')
BT_train_test_oot_fdr(classifier, N=10, var_num=30)

1th fold
	Feature Importance:	
[0.11842395 0.00787727 0.00187314 0.         0.00371849 0.00125998
 0.         0.00183643 0.01835708 0.00353939 0.00285689 0.0038089
 0.00338466 0.0039411  0.11622768 0.00968711 0.00406235 0.00114173
 0.00172507 0.00396627 0.001792   0.6774872  0.00136176 0.
 0.00200718 0.00140617 0.         0.00134904 0.0054773  0.00143185]
	FDR_train: 0.5905001451097998
	FDR_test: 0.556135770234987
	FDR_oot: 0.5553227158424141

2th fold
	Feature Importance:	
[0.09091051 0.00562613 0.00134827 0.         0.00306657 0.00102713
 0.         0.00126026 0.01219592 0.00220983 0.00195543 0.00274204
 0.00248254 0.0023543  0.10472185 0.00668988 0.00281299 0.00079196
 0.00128893 0.0030769  0.00141148 0.7419015  0.00095847 0.
 0.00142188 0.00108917 0.         0.00097827 0.00459321 0.00108462]
	FDR_train: 0.595143658701751
	FDR_test: 0.515230635335074
	FDR_oot: 0.5532271584241408

3th fold
	Feature Importance:	
[0.12027524 0.00685483 0.0018161  0.         0.00500676 0.00129983
 0.   